In [1]:
import numpy as np 
import pandas as pd 
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import string
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model
import tensorflow as tf

In [2]:
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')
stopwords = set(stopwords.words('english'))

In [3]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
rawData = pd.read_csv('/kaggle/input/customer-support-on-twitter/twcs/twcs.csv')
company = "AppleSupport"
rawData.head(5)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [5]:
rawData[rawData['author_id'].str.contains(company)].head(30)

companyAnswers = rawData[rawData['author_id'].str.contains(company)].head(1000)

companyAnswers.head(5)

answerTweets = []
for idx, t in companyAnswers[companyAnswers['author_id'].str.contains(company)].iterrows():
     if not np.isnan(t['in_response_to_tweet_id']):
 
        answerTweets.append(t)

questionTweets = []

answerTweets[0]['in_response_to_tweet_id']

for a in answerTweets:
    question = rawData.loc[rawData['tweet_id'] == a['in_response_to_tweet_id']]
    questionTweets.append(question['text'].to_string(index=False))

for idx, t in enumerate(answerTweets):
    answerTweets[idx] = answerTweets[idx]['text']
    
print(answerTweets[2])
print(questionTweets[2])
print(len(answerTweets))
print(len(questionTweets))

@115855 Let's go to DM for the next steps. DM us here: https://t.co/GDrqU22YpT
@AppleSupport Tried resetting my settings .. re...
997
997


In [6]:
qListTemp = []
aListTemp = []

for t in questionTweets:
    t = re.sub('@[^\s]+','',t)
    t = re.sub('http[^\s]+','',t)
    qListTemp.append(t)
    
for t in answerTweets:
    t = re.sub('@[^\s]+','',t)
    t = re.sub('http[^\s]+','',t)
    aListTemp.append(t)

questionTweets = qListTemp
answerTweets = aListTemp
pairs = list(zip(questionTweets,answerTweets))
print(print(pairs[66]))

(' really needs to fix this update. My pho...', " We're here to help with any issues you may having with performance. Let us know the exact iOS version to get started.")
None


In [7]:
input_docs = []
target_docs = []
input_tokens = set()
target_tokens = set()

for tweet in pairs:
    input_doc, target_doc = tweet[0], tweet[1] 
    input_docs.append(input_doc)

    target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))

    target_doc = '<START> ' + target_doc + ' <END>' 
    target_docs.append(target_doc)

    for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
        if token not in input_tokens:
            input_tokens.add(token)
    for token in target_doc.split():
        if token not in target_tokens:
            target_tokens.add(token)

input_tokens = sorted(list(input_tokens))
print("INPUT TOKENS")
print(input_tokens)

target_tokens = sorted(list(target_tokens))
print("TARGET TOKENS")
print(target_tokens)

num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

input_features_dict = dict([(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict([(token, i) for i, token in enumerate(target_tokens)])

print("INPUT FEATURES")
print(input_features_dict)

reverse_input_features_dict = dict((i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict((i, token) for token, i in target_features_dict.items())

max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])

encoder_input_data = np.zeros((len(input_docs), max_encoder_seq_length, num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros((len(input_docs), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(input_docs), max_decoder_seq_length, num_decoder_tokens),dtype='float32')

print("encoder input")
print(encoder_input_data)

INPUT TOKENS
['!', '"', '#', '$', '%', '&', '(', ')', '*', '+', ',', '-', '.', '/', '0', '000', '03', '1', '10', '100', '100335493202', '11', "11'", '12', '13', '14', '15', '15B150', '18', '1h', '1にあげたらTwitter少し見てるだけだし低電力モードなのに20パーへったぞ', '2', '20', '2010', '2011', '23', '256', '28', '2nd', '3', '30', '38', '3D', '3rd', '4', '42', '4times', '5', '5S', '5s', '6', '60', '6S', '6SPlus', '6s', '6sPlus', '7', '7plus', '8', '9TB', ':', ';', '=', '?', 'A', 'ACONTECENDO', 'AF', 'AFTER', 'AGAIN', 'AND', 'APPLE', 'AQUI', 'ASA', 'AT', 'Actualize', 'Add', 'After', 'AirPods', 'Akku', 'Aktivierung', 'Alguém', 'All', 'Already', 'Also', 'Although', 'Amma', 'And', 'Animal', 'Annnnnnd', 'Another', 'Any', 'Anyone', 'App', 'Apple', 'AppleCa', 'AppleWatch', 'Applewatch', 'At', 'Auto', 'Ayo', 'Ayúdame', 'BLUETOOTH', 'BRAND', 'BRO', 'BUTTON', 'Baje', 'Basically', 'Battery', 'Bec', 'Been', 'Benutzung', 'Bienvenida', 'Bl', 'Bluetoo', 'Bluetooth', 'Both', 'Bridgeport', 'Bro', 'But', 'C', 'CONSTANTLY', 'Ca', 'Cal

In [8]:
#positional encoding
for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):
    for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
        encoder_input_data[line, timestep, input_features_dict[token]] = 1. 

    
    for timestep, token in enumerate(target_doc.split()):
        decoder_input_data[line, timestep, target_features_dict[token]] = 1. 
        if timestep > 0: 
            decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.
print(pairs[:10])

[('  ', " We're here for you. Which version of the iOS are you running? Check from Settings &gt; General &gt; About."), (' The newest update. I️ made sure t...', " Lets take a closer look into this issue. Select the following link to join us in a DM and we'll go from there. "), (' Tried resetting my settings .. re...', " Let's go to DM for the next steps. DM us here: "), (' This is what it looks like ', ' Any steps tried since it started last night?'), (' I️ have an iPhone 7 Plus and yes ...', " That's great it has iOS 11.1 as we can rule out being outdated. Any steps tried since this started? Do you recall when it started?"), (' I️ need answers because it’s anno...', " We'd like to look into this with you. Which model do you have and is iOS 11.1 installed? Any steps tried so far?"), ('Hey  and anyone else who upgraded ...', " Hey, let's work together to figure out what's going on. Meet us in DM and we'll continue from there. "), (' This is what is happening... ', " We'd like to invest

In [9]:
dimensionality = 256 
batch_size = 10 
epochs = 600

#Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]

#Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#Model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

#Compiling
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')

#Training
training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)


2022-12-23 09:23:11.991971: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-23 09:23:12.136209: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-23 09:23:12.136978: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-23 09:23:12.138479: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/600


2022-12-23 09:23:20.430535: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


80/80 [==============================] - 8s 34ms/step - loss: 1.8193 - accuracy: 0.0247 - val_loss: 1.6815 - val_accuracy: 0.0262
Epoch 2/600
80/80 [==============================] - 2s 20ms/step - loss: 1.7600 - accuracy: 0.0270 - val_loss: 1.6687 - val_accuracy: 0.0256
Epoch 3/600
80/80 [==============================] - 2s 22ms/step - loss: 1.7428 - accuracy: 0.0277 - val_loss: 1.6655 - val_accuracy: 0.0279
Epoch 4/600
80/80 [==============================] - 2s 21ms/step - loss: 1.7306 - accuracy: 0.0285 - val_loss: 1.6667 - val_accuracy: 0.0264
Epoch 5/600
80/80 [==============================] - 2s 22ms/step - loss: 1.7204 - accuracy: 0.0287 - val_loss: 1.6659 - val_accuracy: 0.0289
Epoch 6/600
80/80 [==============================] - 2s 21ms/step - loss: 1.7136 - accuracy: 0.0291 - val_loss: 1.6662 - val_accuracy: 0.0273
Epoch 7/600
80/80 [==============================] - 2s 23ms/step - loss: 1.7034 - accuracy: 0.0292 - val_loss: 1.6734 - val_accuracy: 0.0340
Epoch 8/600
80/80 

In [10]:
training_model.save('training_model.h5')

In [11]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [12]:
from keras.models import load_model
training_model = load_model('training_model.h5')
encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

latent_dim = 256
decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

def decode_response(test_input):
    states_value = encoder_model.predict(test_input)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_features_dict['<START>']] = 1.
    
    decoded_sentence = ''

    stop_condition = False
    
    while not stop_condition:
        output_tokens, hidden_state, cell_state = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_features_dict[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if (sampled_token == '<END>' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        states_value = [hidden_state, cell_state]
    return decoded_sentence

In [13]:
name = "100557155"

testQuestions = [
    "Hello",
    "since updating I am unable to use mail becuase of a bug with the I key",
    "I have been trying to conact the call centres what time are they open?",
    "I need to upgrade my phone because my battery life is bad",
    "After I upgraded my phone, I am unable to get past the lock screen",
    "I cannot use my home button for some reason, please help",
    "@AppleSupport The newest update. I️ made sure to download it yesterday", #this question is directly copied from dataset
    "Thanks for helping me out!",
    "bye"
    
]
class ChatBot:
    negative_responses = ("no", "nope", "nah", "naw", "not a chance", "sorry")
    exit_commands = ("quit", "pause", "exit", "goodbye", "bye", "later", "stop")
    def start_chat(self):
    for q in testQuestions:
        print(name + ": " + q)
        print("BOT: " + self.generate_response(q))
    
    def chat(self, reply):
        while not self.make_exit(reply):
        reply = input(self.generate_response(reply)+"\n")
    
    def string_to_matrix(self, user_input):    
        tokens = re.findall(r"[\w']+|[^\s\w]", user_input)
        user_input_matrix = np.zeros((1, max_encoder_seq_length, num_encoder_tokens),dtype='float32')
        for timestep, token in enumerate(tokens):
            if token in input_features_dict:
                user_input_matrix[0, timestep, input_features_dict[token]] = 1.
    return user_input_matrix
  
    def generate_response(self, user_input):
        input_matrix = self.string_to_matrix(user_input)
        chatbot_response = decode_response(input_matrix)
        chatbot_response = chatbot_response.replace("<START>",'')
        chatbot_response = chatbot_response.replace("<END>",'')
    return chatbot_response

    def make_exit(self, reply):
        for exit_command in self.exit_commands:
            if exit_command in reply:
                print("Ok, have a great day!")
                return True
        return False
  
chatbot = ChatBot()
chatbot.start_chat()

100557155: Hello
BOT:  We've received your DM and will continue there . . . . . . . . . . . . .
100557155: since updating I am unable to use mail becuase of a bug with the I key
BOT:  We can help ! What happens after it says verifying . Is that when it restarts
100557155: I have been trying to conact the call centres what time are they open?
BOT:  We want to help you . That station may not be available . Availability of
100557155: I need to upgrade my phone because my battery life is bad
BOT:  We want to help you . That station may not be available . Availability of
100557155: After I upgraded my phone, I am unable to get past the lock screen
BOT:  Send us a DM and we'll continue there . . . . . . . . . . . . . . . . .
100557155: I cannot use my home button for some reason, please help
BOT:  If you're still having troubles , be sure to follow up with our team that
100557155: @AppleSupport The newest update. I️ made sure to download it yesterday
BOT:  We're here to help . Which version 